In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time
import torch

### скачаем все необходимое

In [2]:
df = pd.read_csv('vacs_hh/resume_train.csv', delimiter=';')
model = SentenceTransformer('AlexWortega/hh_search')

No sentence-transformers model found with name /home/zjkarina/.cache/torch/sentence_transformers/AlexWortega_hh_search. Creating a new one with MEAN pooling.


[2023-12-24 11:36:41,150] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-12-24 11:36:46.155025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-24 11:36:47.217904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### замерим время кодирования датасета в 10000 примеров

In [3]:
job_descriptions = list(df['position'])
start = time.time()
# Encode the job descriptions
job_embeddings = model.encode(job_descriptions, convert_to_tensor=True)
all = time.time() - start

In [4]:
all

68.85204982757568

### Замерим поиск топ-1 близкого вектора

In [7]:
start = time.time()
query_embedding = model.encode('Надзор за выполнением проектных работ.нтации для разработки проекта, технических условий на инженерное обеспечение проектируемого объекта, разрешения на подключение к источникам энергоснабжения', convert_to_tensor=True).to(model.device)
cos_scores = util.cos_sim(query_embedding, job_embeddings)[0]
top_results = torch.topk(cos_scores, k=1)
all = time.time() - start

In [8]:
indices = [i.item() for i in top_results.indices]
for res in df['position'][indices]:
    print(res)

Начальник отдела обследований и проектирования


In [9]:
all

0.039879560470581055

### Замерим поиск топ-5 близкого вектора

In [10]:
start = time.time()
query_embedding = model.encode('Надзор за выполнением проектных работ.нтации для разработки проекта, технических условий на инженерное обеспечение проектируемого объекта, разрешения на подключение к источникам энергоснабжения', convert_to_tensor=True).to(model.device)
cos_scores = util.cos_sim(query_embedding, job_embeddings)[0]
top_results = torch.topk(cos_scores, k=5)
all = time.time() - start

In [11]:
all

0.023413419723510742

In [ ]:
indices = [i.item() for i in top_results.indices]
for res in df['position'][indices]:
    print(res)